---
# MLlib Spark - Classificação

## Aprendizado de Máquina no Apache Spark

Alunos:


*   **Alisson Nunes** (725862)
*   **Lucas Mathaeus** (726561)
*   **William Eugênio** (726601)

Bacharelado em Ciência da Computação (BCC)<br>
Departamento de Computação (DC)<br>
Universidade Federal de São Carlos (UFSCar)


# (0) Contextualização

## Aprendizado de Máquina

- .
- .

## Classificação

## Resilient Distributed Dataset (RDD)

#  (1) Instalação e Configuração

In [1]:
! pip install -q pyspark numpy

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 50.0 MB/s 


In [2]:
# Configurando as variaveis de ambiente
import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [3]:
# Definindo uma sessão e contexto Spark
from pyspark.sql import SparkSession
ss = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()
sc = ss.sparkContext

# (2) Obtenção dos Dados

In [15]:
# Instalando o módulo wget
%%capture
! pip install -q wget
!rm -rf data
!mkdir data

In [6]:
# Baixando os dados de exemplo para classificação
import wget

url = "https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_binary_classification_data.txt"
wget.download(url, "data/sample_binary_classification_data.txt")

'data/sample_binary_classification_data.txt'

In [7]:
# Carregando o arquivo de dados
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, "data/sample_binary_classification_data.txt")

# (3) Tratamento dos Dados

In [8]:
training, test = data.randomSplit([0.6, 0.4], seed=11)
training.cache()

PythonRDD[4] at RDD at PythonRDD.scala:53

In [9]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

model = LogisticRegressionWithLBFGS.train(training)

In [10]:
predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

In [11]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

metrics = BinaryClassificationMetrics(predictionAndLabels)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [12]:
print("Area under PR = %s" % metrics.areaUnderPR)
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 1.0
Area under ROC = 1.0


In [13]:
print(predictionAndLabels.take(10))

[(1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 0.0), (1.0, 1.0)]


In [14]:
sc.stop()